In [79]:
from couplets_utils import *
from keras.layers import Input, Dense, LSTM, Reshape, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np

In [3]:
sample = load_sample_datasets()
sample_X = sample['sample_X']
sample_Y = sample['sample_Y']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [5]:
print('sample_X:', sample_X.shape)
print('sample_Y:', sample_Y.shape)

sample_X: (100, 30)
sample_Y: (100, 30)


In [51]:
def seq2onehot(x):
    x = K.cast(x, 'int32')
    return K.one_hot(x, vocabs_size)

In [110]:
def onehot2seq(x):
    return K.argmax(x)

In [116]:
onehot = seq2onehot(sample_X)
onehot

<tf.Tensor 'one_hot_8:0' shape=(100, 30, 5000) dtype=float32>

In [118]:
onehot2seq(onehot[0])

<tf.Tensor 'ArgMax_2:0' shape=(30,) dtype=int64>

In [123]:
def train_model(n_x, n_a, Tx):
    input = Input(shape=(Tx,), name='x0')
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')

    x = Lambda(seq2onehot)(input)
        
    a = a0
    c = c0
    
    lstm_cell = LSTM(units=n_a, return_state=True)
    dense_layer = Dense(units=n_x, activation='softmax')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[i])(x)
        x = Reshape(target_shape=(1, -1))(x)
        a, _, c = lstm_cell(x, initial_state=[a, c])
        x = dense_layer(a)
        x = Reshape(target_shape=(1, -1))(x)
        outputs.append(x)
        
    
        
    outputs = Lambda(onehot2seq)(outputs)
            
    model = Model(inputs=[input, a0, c0], outputs=outputs)
        
    return model

In [124]:
train_model = train_model(vocabs_size, 16, max_len)

TypeError: 'Variable' object does not support item assignment

In [97]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [101]:
sample_X.shape

(100, 30)

In [104]:
a0 = np.zeros((100, 16))
c0 = np.zeros((100, 16))
train_model.fit([sample_X, a0, c0], list(sample_Y), batch_size=8, epochs=2)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 30 array(s), but instead got the following list of 100 arrays: [array([[ 155],
       [  68],
       [ 302],
       [ 367],
       [1672],
       [ 730],
       [   2],
       [  55],
       [  10],
       [ 261],
       [ 874],
       [ 263],
       [ 574],
    ...